In [18]:
pip install mkgendocs

Note: you may need to restart the kernel to use updated packages.


In [27]:

import ast
import importlib
import re
import json, yaml
from collections import defaultdict
from pathlib import Path
from typing import Union, get_type_hints
import os
import shutil


from mkgendocs.gendocs import generate
from mkgendocs.parse import GoogleDocString

from mako.template import Template

In [20]:
os.getcwd()

'/home/andrea/Code/ProteinModels'

In [21]:
#read in the  file header
script = '/home/andrea/Code/ProteinModels/protml/apps/train.py'
repo_dir = '/home/andrea/Code/ProteinModels'
mkdocs_dir = '/home/andrea/Code/ProteinModels/docs' 

intro_contents = defaultdict(dict)
with open(script, 'r') as source:
    tree = ast.parse(source.read())

intros = []
for child in ast.iter_child_nodes(tree):
    try:
        if isinstance(child, (ast.Expr)):
            child.value.s
            intros.append(child.value.s)
    except:
        pass

In [22]:
docstring = intros[0]
docstring_parser = GoogleDocString(docstring)

In [23]:
docstring_parser.parse()

[{'header': '',
  'text': 'The *protml app* for training models to map protein sequences to their phenotype and\ngenerative models for generate sequences with high functional scores  \n',
  'args': []},
 {'header': 'Examples',
  'text': 'Train a supervised model:\n    ```python\n    python3 -m protml.apps.train experiment=supervised/train_base                 train_data= < PATH_TO_TRAINING_DATA >                val_data= < PATH_TO_VALIDATION_DATA > \n    ```\nOveride model parameters from the command line:\n\n    python3 -m protml.apps.train experiment=supervised/train_base                 train_data= < PATH_TO_TRAINING_DATA >                val_data= < PATH_TO_VALIDATION_DATA >                trainer.max_epochs=50000                model.encoder.model_params.hidden_layer_sizes=[100,100,100,100,100]\\ \n            z_dim=10    \n\nTrain a generative model:\n\n    python3 -m protml.apps.train experiment=vae/train_base                 train_data=< PATH_TO_TRAINING_DATA >                v

In [17]:
headers, data = docstring_parser.markdown()
'signature' in data[1]

False

In [43]:
format_md_text(data[1]['text'])

['Train a supervised model:', '', 'python3 -m protml.apps.train experiment=supervised/train_base train_data= < PATH_TO_TRAINING_DATA > val_data= < PATH_TO_VALIDATION_DATA >', '', 'Overide model parameters from the command line:', '', 'python3 -m protml.apps.train experiment=supervised/train_base train_data= < PATH_TO_TRAINING_DATA > val_data= < PATH_TO_VALIDATION_DATA > trainer.max_epochs=50000 model.encoder.model_params.hidden_layer_sizes=[100,100,100,100,100]\\', 'z_dim=10', '', 'Train a generative model:', '', 'python3 -m protml.apps.train experiment=vae/train_base train_data=< PATH_TO_TRAINING_DATA > val_data= <PATH_TO_VALIDATION_DATA > trainer.max_epochs=1000', '', 'Specify parameters that are not set in the config file with +PARAMETER=VALUE:', '', 'python3 -m protml.apps.train experiment=vae/train_base train_data=< PATH_TO_TRAINING_DATA > val_data= <PATH_TO_VALIDATION_DATA > trainer.max_epochs=1000 +datamodule.params.use_weights=True', '', '']


In [54]:
import re
#function to format the text section of the docstring data and extract code blocks
def format_md_text(text):
    lines = text.split('\n')
    python_lines = []

    for i,line in enumerate(lines):
        #cleanup lines
        line = line.strip()
        line = re.sub(r'\s+', ' ', line)
        
        if re.match(r'^python( 3)? ', line):
            line = f'```python\n{line}\n```'
        lines[i] = line
    
    return '\n'.join(lines)
        

def data_to_markdown(data):
    """Converts the data from the docstring parser to markdown"""
    markdown = []
    for d in data:
        if 'header' in d and len(d['header']) > 0:
            markdown.append(f'### {d["header"]}')
        if 'signature' in d:
            markdown.append(f'```python\n {d["signature"]}\n```')
        if 'description' in d:
            markdown.append(d['description'])
        if 'args' in d and len(d['args']) > 0:
            markdown.append('### Arguments')
            for arg in d['args']:
                markdown.append(f'* **{arg["name"]}** ({arg["type"]}): {arg["description"]}')
        if 'returns' in d:
            markdown.append('### Returns')
            markdown.append(f'* **{d["returns"]["type"]}**: {d["returns"]["description"]}')
        if 'raises' in d:
            markdown.append('### Raises')
            for r in d['raises']:
                markdown.append(f'* **{r["type"]}**: {r["description"]}')
        if 'attributes' in d:
            markdown.append('### Attributes')
            for attr in d['attributes']:
                markdown.append(f'* **{attr["name"]}** ({attr["type"]}): {attr["description"]}')
        if 'text' in d:
            markdown.append(format_md_text(d['text']))

    return '\nn'.join(markdown)


In [56]:
markdown_str = data_to_markdown(data)
print(markdown_str)

The *protml app* for training models to map protein sequences to their phenotype and
generative models for generate sequences with high functional scores

n### Examples
nTrain a supervised model:

python3 -m protml.apps.train experiment=supervised/train_base train_data= < PATH_TO_TRAINING_DATA > val_data= < PATH_TO_VALIDATION_DATA >

Overide model parameters from the command line:

python3 -m protml.apps.train experiment=supervised/train_base train_data= < PATH_TO_TRAINING_DATA > val_data= < PATH_TO_VALIDATION_DATA > trainer.max_epochs=50000 model.encoder.model_params.hidden_layer_sizes=[100,100,100,100,100]\
z_dim=10

Train a generative model:

python3 -m protml.apps.train experiment=vae/train_base train_data=< PATH_TO_TRAINING_DATA > val_data= <PATH_TO_VALIDATION_DATA > trainer.max_epochs=1000

Specify parameters that are not set in the config file with +PARAMETER=VALUE:

python3 -m protml.apps.train experiment=vae/train_base train_data=< PATH_TO_TRAINING_DATA > val_data= <PATH_TO_VA

In [25]:
DOCSTRING_TEMPLATE = """
## if we are processing a header function
%if header['Examples']:
${h3} .${header['Examples']}
%endif 

%for section in sections:
    %if section['header']:

**${section['header']}**

    %else:
---
    %endif
    %if section['args']:
        %for arg in section['args']:
        %if arg['field']:
* **${arg['field']}** ${arg['signature']} : ${arg['description']}
        %else:
* ${arg['description']}
        %endif
        %endfor
    %endif
${section['text']}
%endfor
"""

In [33]:
target_info = {'name': 'protml', 'signature': 'python3', 'docstring':docstring}

In [28]:
docstring_template = DOCSTRING_TEMPLATE
markdown_template = Template(text=docstring_template)

In [35]:
markdown_str = markdown_template.render(header=data[1],
                                       source=None,
                                       signature='python3',
                                       sections=data,
                                       headers=headers,
                                       h2='##', h3='###')

KeyError: 'Examples'

Experiment Description
You should design a set of experiments in order to evaluate your proposed methods.

# 3) Experimental Strategy Outline


I will follow the steps outlined below to develop and evaluate a prediction model. Due to limited time and compute resources, these steps are designed to illustrate the project outline, an rather than representing real optimization, and only a small subset of parameters are tested in each case.  

i) use the simple model architecture of the MAVE_NN network as a baseline model to benchmark further development
 
 -  train on single protein datasets to verify the predictive power is similar to the published model
 
- with the view of using a more complex encoder method to map sequences into latent space, extend the measurment models from 1 to more dimensions

ii) Originally,  VAE models for proteins are trained on their probability in evolution. Here we check how a simple VAE model sampling from data distribution with $\exp(-\Delta\DeltaG) structures latent representation, as a better starting point for training a regression head.

iii) Test the performance and generalization of the baseline models for multiple sequence data 

iv) Calculate embedded representation of the sequences from a pretrained protein language model (*T5prot*).

v) Use the embedding vectors of the  as input to the Simple Network model (instead of the one hot encoded sequences used in the reference case) and evaluate the performance

vi) add the wt sequence as an additional input feature. Rationale: jointly embedding different proteins, will likely create representations where sequences with a single AA difference are located close together. Adding the wt sequence we can train the prediction model on the distance from the wt sequence in the embedded space instead. 





# 4) Model Training

In [ ]:
import os 
import sys
import numpy as np
import pandas as pd

import torch
import pytorch_lightning as pl

import h5py
import importlib
import hydra
import omegaconf
from omegaconf import DictConfig


/home/andrea/anaconda3/envs/protml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The models were implemented in protml and trained using the train app of the protml framework.

The run commands for the different experiments are collected in this notebook, and run for a single epoch for illustration. Real model training was done with these on COLAB gpus and as background scripts

### i) Training supervised single protein baseline model 
Model source code: protml/models/baseline_supervised.py

adapted from https://github.com/jbkinney/mavenn

In [ ]:
#Supervised training with the same setup as in the MAVE-NN publication 
!python3 -m protml.apps.train experiment=supervised/train_base \
    train_data=Data/data_sets/df_train_rand_1MJC.csv val_data=Data/data_sets/df_val_rand_1MJC.csv \
        trainer.max_epochs=1 model.encoder.model_params.hidden_layer_sizes=[100,100,100,100,100] 

[2023-05-09 21:25:23,053][__main__][INFO] - Instantiating datamodule <protml.dataloaders.SequenceDataModule>
[2023-05-09 21:25:23,071][__main__][INFO] - Seeding with 12345
Global seed set to 12345
[2023-05-09 21:25:23,073][__main__][INFO] - Instantiating model <protml.models.ENC_M.factory>
[2023-05-09 21:25:23,101][__main__][INFO] - Instantiating logger <pytorch_lightning.loggers.MLFlowLogger>
[2023-05-09 21:25:23,209][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.ModelCheckpoint>
[2023-05-09 21:25:23,213][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.EarlyStopping>
[2023-05-09 21:25:23,214][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.RichModelSummary>
[2023-05-09 21:25:23,214][__main__][INFO] - Instantiating trainer <pytorch_lightning.Trainer>
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSumm

In [ ]:
!python3 -m protml.apps.train experiment=supervised/train_base \
    train_data=Data/data_sets/df_train_rand_1MJC.csv val_data=Data/data_sets/df_val_rand_1MJC.csv \
        trainer.max_epochs=10 model.encoder.model_params.hidden_layer_sizes=[100,100,100,100,100] 

[2023-05-09 21:26:06,702][__main__][INFO] - Instantiating datamodule <protml.dataloaders.SequenceDataModule>
[2023-05-09 21:26:06,716][__main__][INFO] - Seeding with 12345
Global seed set to 12345
[2023-05-09 21:26:06,717][__main__][INFO] - Instantiating model <protml.models.ENC_M.factory>
[2023-05-09 21:26:06,732][__main__][INFO] - Instantiating logger <pytorch_lightning.loggers.MLFlowLogger>
[2023-05-09 21:26:06,814][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.ModelCheckpoint>
[2023-05-09 21:26:06,817][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.EarlyStopping>
[2023-05-09 21:26:06,817][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.RichModelSummary>
[2023-05-09 21:26:06,818][__main__][INFO] - Instantiating trainer <pytorch_lightning.Trainer>
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSumm

In [ ]:
#increased layer size 
!python3 -m protml.apps.train experiment=supervised/train_base \
    train_data=Data/data_sets/df_train_rand_1MJC.csv val_data=Data/data_sets/df_val_rand_1MJC.csv \
        trainer.max_epochs=20 model.encoder.model_params.hidden_layer_sizes=[200,200,200,200,200] z_dim=10 seed=42

[2023-05-09 21:27:22,844][__main__][INFO] - Instantiating datamodule <protml.dataloaders.SequenceDataModule>
[2023-05-09 21:27:22,858][__main__][INFO] - Seeding with 42
Global seed set to 42
[2023-05-09 21:27:22,859][__main__][INFO] - Instantiating model <protml.models.ENC_M.factory>
[2023-05-09 21:27:22,876][__main__][INFO] - Instantiating logger <pytorch_lightning.loggers.MLFlowLogger>
[2023-05-09 21:27:22,953][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.ModelCheckpoint>
[2023-05-09 21:27:22,956][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.EarlyStopping>
[2023-05-09 21:27:22,957][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.RichModelSummary>
[2023-05-09 21:27:22,957][__main__][INFO] - Instantiating trainer <pytorch_lightning.Trainer>
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` c

In a more indepth analysis, many more model options and architecturs could be evaluated easily, by looking more extensively at different parameter combinations, and bringing ray tune into the framework. One would also look at other single protein sets. The system is well suited for parameter optimization, as the protein sizes are small and training runs fast.  

### ii) Train a baseline VAE model with free energy based probability sampling
model source code: protml/models/baseline_vae.py


*Adapted from the EVE model: https://github.com/OATML/EVE* 

In [ ]:
#Training a basic  VAE with a latent spce dimension of z=50 as recomended in the EVE pubpicatin, as well as z=10 to make it compatible with 
#the MAVE model that is designed for a low dimensional representation. 

# !Note: the datamodule parameter: +datamodule.params.use_weights=True switches on the sample weights to exp(-DDG). 

!python3 -m protml.apps.train experiment=vae/train_base \
    train_data=Data/data_sets/df_train_rand_1MJC.csv val_data=Data/data_sets/df_val_rand_1MJC.csv \
        trainer.max_epochs=1 model.optimizer.lr=1e-4 model.encoder.model_params.hidden_layer_sizes=[1000,500,100]\
        model.decoder.model_params.hidden_layer_sizes=[100,500,1000] z_dim=50 seed=42 +datamodule.params.use_weights=True\
          callbacks.early_stopping.patience=1000
        
!python3 -m protml.apps.train experiment=vae/train_base \
    train_data=Data/data_sets/df_train_rand_1MJC.csv val_data=Data/data_sets/df_val_rand_1MJC.csv \
        trainer.max_epochs=1 model.optimizer.lr=1e-4 model.encoder.model_params.hidden_layer_sizes=[1000,500,100]\
        model.decoder.model_params.hidden_layer_sizes=[100,500,1000] z_dim=10 seed=42 +datamodule.params.use_weights=True\
       

[2023-05-09 22:14:02,719][__main__][INFO] - Instantiating datamodule <protml.dataloaders.SequenceDataModule>
[2023-05-09 22:14:02,750][__main__][INFO] - Seeding with 42
Global seed set to 42
[2023-05-09 22:14:02,752][__main__][INFO] - Instantiating model <protml.models.VAE>
[2023-05-09 22:14:02,846][__main__][INFO] - Instantiating logger <pytorch_lightning.loggers.MLFlowLogger>
[2023-05-09 22:14:02,997][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.ModelCheckpoint>
[2023-05-09 22:14:03,002][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.EarlyStopping>
[2023-05-09 22:14:03,003][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.RichModelSummary>
[2023-05-09 22:14:03,004][__main__][INFO] - Instantiating trainer <pytorch_lightning.Trainer>
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
G

### iii) Testing generalization of the baseline Model to the whole dataset

In [ ]:
!python3 -m protml.apps.train experiment=supervised/train_base \
    train_data=Data/data_sets/train_data_rand.csv val_data=Data/data_sets/val_data_rand.csv \
        trainer.max_epochs=1 model.encoder.model_params.hidden_layer_sizes=[100,100,100,100,100] \
         z_dim=1 callbacks.early_stopping.patience=500  callbacks.model_checkpoint.save_last=last  

!python3 -m protml.apps.train experiment=supervised/train_base \
    train_data=Data/data_sets/train_data_rand.csv val_data=Data/data_sets/val_data_rand.csv \
        trainer.max_epochs=500 model.encoder.model_params.hidden_layer_sizes=[100,100,100,100,100] z_dim=10 seed=42 trainer=lightning/gpu

[2023-05-09 22:30:15,998][__main__][INFO] - Instantiating datamodule <protml.dataloaders.SequenceDataModule>
[2023-05-09 22:30:16,029][__main__][INFO] - Seeding with 12345
Global seed set to 12345
[2023-05-09 22:30:16,031][__main__][INFO] - Instantiating model <protml.models.ENC_M.factory>
[2023-05-09 22:30:16,063][__main__][INFO] - Instantiating logger <pytorch_lightning.loggers.MLFlowLogger>
[2023-05-09 22:30:16,237][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.ModelCheckpoint>
[2023-05-09 22:30:16,242][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.EarlyStopping>
[2023-05-09 22:30:16,243][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.RichModelSummary>
[2023-05-09 22:30:16,244][__main__][INFO] - Instantiating trainer <pytorch_lightning.Trainer>
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSumm

### iv) Calculating the sequence embedding vectors from the pretrained T5Prot protein language model

! Use a GPU for this part. It is computationally expensive. 

ProtTrans Language Model: https://github.com/agemagician/ProtTrans

Scripts and functions are adapted from https://github.com/Rostlab/VESPA

... 

In [ ]:
# Import dependencies and check whether GPU is available. 
from transformers import T5EncoderModel, T5Tokenizer, T5ForConditionalGeneration
import torch
import h5py
import time

#ensure that GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using {}".format(device))

Using cpu


Define the functions for generating the embedding

In [ ]:
# Load ProtT5 in half-precision (more specifically: the encoder-part of ProtT5-XL-U50) 
def get_T5_model():
    model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")
    model = model.to(device) # move model to GPUif available
    #model = model.eval() # set model to evaluation model
    tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)

    return model, tokenizer



def get_embeddings( model, tokenizer, seqs:list[tuple], per_residue:bool, per_protein:bool, max_length = 73, 
                   max_residues=4000, max_seq_len=1000, max_batch=100 ) -> dict:
    """  Generate embeddings via batch-processing

    Args:
        model: pretrained model
        tokenizer: T5model Tokeniser
        seqs (list): Tuples mapping of unique identifier to sequence
        per_residue (bool): indicates that embeddings for each residue in a protein should be returned.
        per_protein (bool): indicates that embeddings for a whole protein should be returned (average-pooling)
        max_length (int, optional): _description_. Defaults to 73.
        max_residues (int, optional): max_residues gives the upper limit of residues within one batch.
        max_seq_len (int, optional): maximum_sequence length. Defaults to 1000.
        max_batch (int, optional): _max_batch gives the upper number of sequences per batch. Defaults to 100.

    Returns:
        dict: embedding results
    """

    results = {"residue_embs" : dict(), 
               "protein_embs" : dict(),
               }
    #already know max length from previous preprocessing
    # sort sequences according to length (reduces unnecessary padding --> speeds up embedding) ...already done that
    #seq_dict   = sorted( seqs.items(), key=lambda kv: len( seqs[kv[0]] ), reverse=True )


    start = time.time()
    batch = list()
    for seq_idx, (pdb_id, seq) in enumerate(seqs,1):
        seq = seq
        seq_len = len(seq)
        seq = ' '.join(list(seq))
        batch.append((pdb_id,seq,seq_len))

        # count residues in current batch and add the last sequence length to
        # avoid that batches with (n_res_batch > max_residues) get processed 
        n_res_batch = sum([ s_len for  _, _, s_len in batch ]) + seq_len 
        if len(batch) >= max_batch or n_res_batch>=max_residues or seq_idx==len(seqs) or seq_len>max_seq_len:
            pdb_ids, seqs, seq_lens = zip(*batch)
            batch = list()

            # add_special_tokens adds extra token at the end of each sequence - this requires a max length of sequence length+1
            token_encoding = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding='max_length', max_length= max_length)
            input_ids      = torch.tensor(token_encoding['input_ids']).to(device)
            attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)
            
            try:
                with torch.no_grad():
                    # returns: ( batch-size x max_seq_len_in_minibatch x embedding_dim )
                    embedding_repr = model(input_ids, attention_mask=attention_mask)
            except RuntimeError:
                print("RuntimeError during embedding for {} (L={})".format(pdb_id, seq_len))
                continue

            for batch_idx, identifier in enumerate(pdb_ids): # for each protein in the current mini-batch
                #f batch_idx % 100 ==0:
                #print('processig batch ', batch_idx )
                s_len = seq_lens[batch_idx]
                emb = embedding_repr.last_hidden_state[batch_idx,:]
               
                if per_residue: # store per-residue embeddings (Lx1024)
                    results["residue_embs"][ identifier ] = emb.detach().cpu().numpy().squeeze()
                if per_protein: # apply average-pooling to derive per-protein embeddings (1024-d)
                    protein_emb = emb[:s_len].mean(dim=0)
                    results["protein_embs"][identifier] = protein_emb.detach().cpu().numpy().squeeze()


    passed_time=time.time()-start
    avg_time = passed_time/len(results["residue_embs"]) if per_residue else passed_time/len(results["protein_embs"])
    print('\n############# EMBEDDING STATS #############')
    print('Total number of per-residue embeddings: {}'.format(len(results["residue_embs"])))
    print('Total number of per-protein embeddings: {}'.format(len(results["protein_embs"])))
    print("Time for generating embeddings: {:.1f}[m] ({:.3f}[s/protein])".format(
        passed_time/60, avg_time ))
    print('\n############# END #############')
    return results


def create_protein_batches(df:pd.DataFrame, protein_ids:list[str])->dict:
    """generates single protein input batches, that can be processed and saved in .h5 file format.
        Required because for processing large batches RAM limit is quickly exceeded.  

    Args:
        df (pd.DataFrame): input dataframe
        protein_ids (list[str]): list of protein name str

    Returns:
        dict: mapping protein names to sequence data
    """
    seq_lists ={}
    for pid in protein_ids:
        seq_dl = []
        #create unique id for each sequ and add to datalist  
        for i,r in df[df.pdbid == pid].iloc[:].iterrows(): 
            seq_dl.append((r.pdbid +'_'+ str(i), r.seq))
        seq_lists[pid] = seq_dl
    return seq_lists

 
def save_embeddings(output_path:str, emb_dict:dict, saving_pattern = 'a')->None:
    """append results to .h5 lookup file, to free memory

    Args:
        output_path (str): h5 file
        emb_dict (dict): results from get_embeddings
        saving_pattern (str, optional): Defaults to 'a'.
    """
    with h5py.File(output_path, saving_pattern) as hf:
        for sequence_id, embedding in emb_dict.items():
            hf.create_dataset(sequence_id, data=embedding)
  

def compute_embedding(model, tokenizer, seq_dict:dict, output_path:str) ->None:
    """compute embeddings for all proteins in seq_dict

    Args:
        model (_T5model_): pretrained model
        tokenizer (T5 Tokenizer): Tokenizer
        seq_dict (dict): input data. protein_id: seq list
        output_path (str): h5 file
    """
    for prot, seqs in seq_dict.items():
        print('processing protein:' , prot )
        results = get_embeddings(model.eval(), tokenizer, seqs=seqs, per_protein=True, per_residue=True, max_length=73)
    
        save_embeddings(os.path.join(output_path,"residue_embeddings1.h5"), results["residue_embs"])
        save_embeddings(os.path.join(output_path,"protein_embeddings1.h5"), results["protein_embs"])

In [ ]:
#preparing the input data for embedding
data_dir = 'Data/data_sets/'

filename = 'train_data_prot.csv'
df = pd.read_csv(os.path.join(data_dir,filename), index_col=0)

filename = 'val_data_prot.csv'
df_v = pd.read_csv(os.path.join(data_dir,filename), index_col=0)

filename = 'test_data_prot.csv'
df_test = pd.read_csv(os.path.join(data_dir,filename), index_col=0)

filename = 'test_proteins_data.csv'
df_ho = pd.read_csv(os.path.join(data_dir,filename), index_col=0)

#strip padding for embedding
df['seq'] = df.seq.apply(lambda x: x.strip('-'))
df_v['seq'] = df_v.seq.apply(lambda x: x.strip('-'))
df_test['seq'] = df_test.seq.apply(lambda x: x.strip('-'))
df_ho['seq'] = df_ho.seq.apply(lambda x: x.strip('-'))

protein_ids = list(df.pdbid.unique())
protein_v_ids = list(df_v.pdbid.unique())
protein_test_ids = list(df_test.pdbid.unique())
protein_ho_ids = list(df_ho.pdbid.unique())

seq_lists = create_protein_batches(df, protein_ids)
seq_lists1 = create_protein_batches(df_v, protein_v_ids)
seq_lists2 = create_protein_batches(df_test, protein_test_ids)
seq_lists3 = create_protein_batches(df_ho, protein_ho_ids)

In [ ]:
model, tokenizer = get_T5_model()
model.device

device(type='cpu')

In [ ]:
#calculate the embeddings -- use GPU--
compute_embedding(model.eval(),tokenizer, seq_lists, 'drive/MyDrive/protT5/output')

In [ ]:
import json

#similarly also get the embeddings for the wt sequences

#first load wildtype dictionary, that was saved in the data_preprocessing steps
j_file = "Data/wt_sequences.json"

with open(j_file, "r") as fj:
    wt_dict = json.load(fj)
fj.close()
wt_list = [(k, seq) for k, seq in wt_dict.items()]

# get the embedding vectors
wt_emb = get_embeddings(model.eval(), tokenizer, seqs=wt_list, per_protein=True, per_residue=True, max_length=73)
save_embeddings('protT5/output/wt_embeddings.h5' , wt_emb['residue_embs'], 'a', )
ssave_embeddings('protT5/output/wt_embeddings.h5' , wt_emb['residue_embs'], 'a', )


############# EMBEDDING STATS #############
Total number of per-residue embeddings: 304
Total number of per-protein embeddings: 304
Time for generating embeddings: 3.1[m] (0.605[s/protein])

############# END #############


To save computation, the calculated embedding files (protein_embeddings.h5, residue_embeddings.h5, wt_embeddings.h5, wt_prot_embeddings.h5) are provided along with the code.

### v) Training models with pretrained embeddings representation of the input

models source code: 
    - protml/models/baseline_supervised 
    - protml/models/dz_supervised 

datamodule: 
    - ddg data. 
    
The pretrained embedding arrays for the whole datasets do not fit into memory. As a workaround the datamodule saves the list of lookup keys as data. 
A custom torch dataset is defined, which retrieves only the embedding arrays for the current batch from .h5 in the __getitem__ method:


 ```python
 EmbeddingsDataset(TensorDataset) 
 ```
 
```python
def __getitem__(self, index):
        
        #lookup embedding in .h5
        pid = self.ids[index]
        embedding = np.array(self.embeddings[pid])
        embedding = embedding[:-1,:]

        return tuple([embedding] + [tensor[index] for tensor in self.tensors]) 
```


In [ ]:
#train baseline model with the embedded sequence representations
!python3 -m protml.apps.train experiment=supervised/train_emb \
    train_data=Data/data_sets/train_data_rand_emb.csv val_data=Data/data_sets/val_data_rand_emb.csv\
    embeddings=protT5/output/residue_embeddings.h5 trainer.max_epochs=1 \
    model.encoder.model_params.hidden_layer_sizes=[100,100,100,100,100] z_dim=1
    

[2023-05-10 01:01:06,722][__main__][INFO] - Instantiating datamodule <protml.dataloaders.EmbeddingsDataModule>
[2023-05-10 01:01:07,443][__main__][INFO] - Seeding with 12345
Global seed set to 12345
[2023-05-10 01:01:07,445][__main__][INFO] - Instantiating model <protml.models.ENC_M.factory>
[2023-05-10 01:01:07,498][__main__][INFO] - Instantiating logger <pytorch_lightning.loggers.MLFlowLogger>
[2023-05-10 01:01:07,591][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.ModelCheckpoint>
[2023-05-10 01:01:07,594][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.EarlyStopping>
[2023-05-10 01:01:07,595][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.RichModelSummary>
[2023-05-10 01:01:07,595][__main__][INFO] - Instantiating trainer <pytorch_lightning.Trainer>
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSu

###  vi) Adding the wt sequence as additional input  
model source code: protml.models.dz_supervised

In [ ]:
#Training on all sequences for referebnce. This representation is likely to be of more use together with a pretrained 
#embedding method, so that the latent space is already representative.   
!python3 -m protml.apps.train experiment=supervised/train_dz \
    train_data=Data/data_sets/train_data_rand.csv val_data=Data/data_sets/val_data_rand.csv \
        trainer.max_epochs=2 model.encoder.model_params.hidden_layer_sizes=[100,100,100,100,100] z_dim=1 seed=42\
        callbacks.model_checkpoint.save_last=last

[2023-05-09 22:45:35,126][__main__][INFO] - Instantiating datamodule <protml.dataloaders.Sequence_WT_DataModule>
[2023-05-09 22:45:44,509][__main__][INFO] - Seeding with 42
Global seed set to 42
[2023-05-09 22:45:44,513][__main__][INFO] - Instantiating model <protml.models.ENC_M_dz.factory>
[2023-05-09 22:45:44,544][__main__][INFO] - Instantiating logger <pytorch_lightning.loggers.MLFlowLogger>
[2023-05-09 22:45:45,208][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.ModelCheckpoint>
[2023-05-09 22:45:45,214][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.EarlyStopping>
[2023-05-09 22:45:45,215][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.RichModelSummary>
[2023-05-09 22:45:45,216][__main__][INFO] - Instantiating trainer <pytorch_lightning.Trainer>
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSum

In [ ]:
# the same using T5 model embeddings
!python3 -m protml.apps.train experiment=supervised/train_emb_wt \
    train_data=Data/data_sets/train_data_rand_emb.csv val_data=Data/data_sets/val_data_rand_emb.csv\
    embeddings=protT5/output/residue_embeddings.h5 wt_embeddings=protT5/output/wt_embeddings.h5 trainer.max_epochs=1 \
    model.encoder.model_params.hidden_layer_sizes=[100,100,100,100,100] z_dim=1

[2023-05-10 01:02:47,160][__main__][INFO] - Instantiating datamodule <protml.dataloaders.Embeddings_WT_DataModule>
[2023-05-10 01:02:47,187][__main__][INFO] - Seeding with 12345
Global seed set to 12345
[2023-05-10 01:02:47,189][__main__][INFO] - Instantiating model <protml.models.ENC_M_dz.factory>
[2023-05-10 01:02:47,243][__main__][INFO] - Instantiating logger <pytorch_lightning.loggers.MLFlowLogger>
[2023-05-10 01:02:47,340][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.ModelCheckpoint>
[2023-05-10 01:02:47,343][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.EarlyStopping>
[2023-05-10 01:02:47,344][__main__][INFO] - Instantiating callback <pytorch_lightning.callbacks.RichModelSummary>
[2023-05-10 01:02:47,344][__main__][INFO] - Instantiating trainer <pytorch_lightning.Trainer>
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `